In [5]:
#Matt
#install.packages("RCurl")
library(RCurl)
library(tidyverse, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)

#Jensen
library(xml2)
library(rvest)
library(httr)
#install.packages("readxl")
library(readxl)
#install.packages("furrr")
library(furrr)
future::plan(multicore)

#Sujung


Loading required package: future



## Webscrape river flows
Matt

In [2]:
#Matt

row_calc1 <- function(tibble){
    n = as.integer(nrow(tibble))
    data <- tail(tibble, (n-4))
    names(data) <- c('Time','Value(m^3/s)')
    return(data)
}

row_calc2 <- function(tibble){
    n = as.integer(nrow(tibble))
    data <- tail(tibble, (n-4)) %>% select(-X) 
    names(data) <- c('Time','Value(m^3/s)')
    return(data)  
}

opuha <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40TK3&Datasets[0].Calculation=Aggregate&Datasets[0].UnitId=140"))) %>% row_calc2()
"Opuha checkpoint"
rakaia <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge.Master%4068526&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc2()
"Rakaia checkpoint"
waitaki <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40TK6&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1()
"Waitaki checkpoint"
kawarau <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40AX2&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1()                                    
"Kawarau checkpoint"
clutha <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=Monthly&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge.Master%4075207&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc2()                  
"Clutha checkpoint"
mataura <- as_tibble(read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2015-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN5&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))) %>% row_calc1() 
"Mataura checkpoint"
#empty data sets, monowai <- read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2020-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN10&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))                    
#taieri <- read.csv(text = getURL("https://hydrowebportal.niwa.co.nz/Export/BulkExport?DateRange=Custom&StartTime=2020-01-01%2000%3A00&EndTime=2020-12-31%2000%3A00&TimeZone=12&Calendar=CALENDARYEAR&Interval=PointsAsRecorded&Step=1&ExportFormat=csv&TimeAligned=True&RoundData=False&IncludeGradeCodes=False&IncludeApprovalLevels=False&IncludeInterpolationTypes=False&Datasets[0].DatasetName=Discharge%20Value.NRWQN%40DN3&Datasets[0].Calculation=Instantaneous&Datasets[0].UnitId=140"))





[1] "Opuha checkpoint"

[1] "Rakaia checkpoint"

[1] "Waitaki checkpoint"

[1] "Kawarau checkpoint"

[1] "Clutha checkpoint"

[1] "Mataura checkpoint"

## Webscrape hydro stations power output
Jensen

### Information variables

In [6]:
# Main site address
main_url <- "https://www.emi.ea.govt.nz"
# Sub address for electricity generation information
power_url <- "/Wholesale/Datasets/Generation/Generation_MD"
# Sub address for hydrostations fleet
stations_url <- "/Wholesale/Datasets/Generation/Generation_fleet/Existing"
# Sub address for network supply points
points_url <- "/Wholesale/Reports/R_NSPL_DR?_si=v|3"

### Retrieve generation output data

In [7]:
# Acquire html code from sub address
source <- paste(main_url, power_url, sep="") %>% read_html()
"Source checkpoint"
# Identify all links in the given html class
all_links <- source %>% html_nodes(".table") %>% html_elements("a") %>% html_attr("href")
"Links identification checkpoint"
# Identify appropriate urls within links and remove duplicates
filtered_links <- all_links[startsWith(all_links, power_url) & !duplicated(all_links)]
"Links filtered checkpoint"
# Extract date information from filtered links
start_pos <- nchar(power_url) + 2
dates <- substring(filtered_links , start_pos, start_pos + 3) 
"Date extraction checkpoint"
# Set dates wanted
dates_wanted <- seq(from=2015,to=2020)
# Select links containing csv data
selected_links <- filtered_links[dates %in% dates_wanted]
"Links selected checkpoint"
# Extract csv data into dataframes
link <- paste(main_url, selected_links, sep="") 
generation_md <- link %>% future_map_dfr(read_csv, show_col_types = FALSE)
"Generation retrieval checkpoint"

# Filter out non hydro stations
generation_md_hydro <- generation_md %>% filter(Tech_Code == "Hydro")
"Data filter checkpoint"
"RETRIEVAL COMPLETE"

[1] "Source checkpoint"

[1] "Links identification checkpoint"

[1] "Links filtered checkpoint"

[1] "Date extraction checkpoint"

[1] "Links selected checkpoint"

[1] "Generation retrieval checkpoint"

[1] "Data filter checkpoint"

[1] "RETRIEVAL COMPLETE"

In [ ]:
#Sujung

## River flows and hydrostations power output combination

In [10]:
## Hydrostations -> rivers
# waitaki -> Waitaki
# highbank -> Rakaia
matching_gen_codes <- c("highbank", "waitaki")
generation_md %>% filter(Gen_Code %in% matching_gen_codes)

Site_Code,POC_Code,Nwk_Code,Gen_Code,Fuel_Code,Tech_Code,Trading_date,TP1,TP2,TP3,⋯,TP41,TP42,TP43,TP44,TP45,TP46,TP47,TP48,TP49,TP50
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-01,10394.654,10402.396,10400.515,⋯,10770.984,10779.275,10777.726,10787.507,10778.765,10776.972,10780.90,10781.60,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-02,10774.541,10779.167,10776.531,⋯,10287.001,10300.329,10288.844,10294.949,10295.537,10296.272,10308.60,10298.00,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-03,10282.572,10294.136,10296.615,⋯,9656.420,9431.461,9409.842,8952.222,8913.463,8924.997,8928.25,8926.29,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-04,9050.878,9072.595,9059.610,⋯,5953.667,5956.185,5956.146,5497.241,5465.989,5464.470,5466.65,5466.53,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-05,5465.519,4542.917,4492.036,⋯,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-06,0.000,0.000,0.000,⋯,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-07,0.000,0.000,0.000,⋯,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-08,0.000,0.000,0.000,⋯,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,NA,NA
HBK,ASB0661,EASH,highbank,Hydro,Hydro,2020-12-09,0.000,0.000,0.000,⋯,0.000,0.000,0.000,0.000,0.000,0.000,0.00,0.00,NA,NA


In [ ]:
#Graphing code